## MLflow's Model Registry

In [3]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='./mlruns/3', creation_time=1686117052556, experiment_id='3', last_update_time=1686117052556, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', creation_time=1685499389635, experiment_id='2', last_update_time=1685499389635, lifecycle_stage='active', name='nyc-taxi-experiment2', tags={}>,
 <Experiment: artifact_location='./mlruns/1', creation_time=1685069600605, experiment_id='1', last_update_time=1685069600605, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/0', creation_time=1685069558389, experiment_id='0', last_update_time=1685069558389, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=7,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 584315e78baf4b97984e7ae02d2c9078, rmse: 6.7423
run id: 5afbd32c28c040aaa236dcd0b966e54b, rmse: 6.9093
run id: f2a15a15b5554ff99fb8ab18804df4da, rmse: 6.9246


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [15]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = "f2a15a15b5554ff99fb8ab18804df4da"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="NYC-Taxi-models")

Registered model 'NYC-Taxi-models' already exists. Creating a new version of this model...
2023/06/07 01:01:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: NYC-Taxi-models, version 3
Created version '3' of model 'NYC-Taxi-models'.


<ModelVersion: creation_timestamp=1686117708277, current_stage='None', description=None, last_updated_timestamp=1686117708277, name='NYC-Taxi-models', run_id='f2a15a15b5554ff99fb8ab18804df4da', run_link=None, source='./mlruns/2/f2a15a15b5554ff99fb8ab18804df4da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [8]:
client.search_registered_models()

[<RegisteredModel: creation_timestamp=1686117218425, description='', last_updated_timestamp=1686117949510, latest_versions=[<ModelVersion: creation_timestamp=1686117708277, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-06-07', last_updated_timestamp=1686118023630, name='NYC-Taxi-models', run_id='f2a15a15b5554ff99fb8ab18804df4da', run_link=None, source='./mlruns/2/f2a15a15b5554ff99fb8ab18804df4da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>,
  <ModelVersion: creation_timestamp=1686117231475, current_stage='Production', description='', last_updated_timestamp=1686117875553, name='NYC-Taxi-models', run_id='584315e78baf4b97984e7ae02d2c9078', run_link='', source='./mlruns/2/584315e78baf4b97984e7ae02d2c9078/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='NYC-Taxi-models', tags={}>]

In [9]:
model_name = "NYC-Taxi-models"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 2, stage: Production


In [22]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1686117708277, current_stage='Staging', description=None, last_updated_timestamp=1686117949510, name='NYC-Taxi-models', run_id='f2a15a15b5554ff99fb8ab18804df4da', run_link=None, source='./mlruns/2/f2a15a15b5554ff99fb8ab18804df4da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [23]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 2, stage: Production


In [24]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1686117708277, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-06-07', last_updated_timestamp=1686118023630, name='NYC-Taxi-models', run_id='f2a15a15b5554ff99fb8ab18804df4da', run_link=None, source='./mlruns/2/f2a15a15b5554ff99fb8ab18804df4da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename, type):
    if type == "parquet":
        df = pd.read_parquet(filename)
    else:
        df = pd.read_csv(filename)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [2]:
df = read_dataframe(
     './data/green_tripdata_2021-03.parquet',
     'parquet'
)

In [10]:
run_id = "f2a15a15b5554ff99fb8ab18804df4da"

client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/mnt/d/ubuntu_files/MLops-Zoomcamp/experiment_track/preprocessor'

In [11]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [12]:
X_test = preprocess(df, dv)

In [13]:
target = "duration"
y_test = df[target].values

In [16]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 217 ms, sys: 66.7 ms, total: 284 ms
Wall time: 18.2 s


{'rmse': 6.659623830022514}

In [17]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 1min 30s, sys: 441 ms, total: 1min 31s
Wall time: 12.7 s


{'rmse': 6.89138218465848}

In [18]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1686117708277, current_stage='Production', description='The model version 3 was transitioned to Staging on 2023-06-07', last_updated_timestamp=1686201614829, name='NYC-Taxi-models', run_id='f2a15a15b5554ff99fb8ab18804df4da', run_link=None, source='./mlruns/2/f2a15a15b5554ff99fb8ab18804df4da/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>